# Open in Colab
<a href="https://colab.research.google.com/github/aekanun2020/2022-PSU-Data-Science-for-Tourism-Course1/blob/main/Quiz2-PySpark_DataScience_onColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .appName("Neural Network Model") \
   .config("spark.executor.memory", "3gb") \
   .getOrCreate()
   
sc = spark.sparkContext

In [ ]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

# 2. Data Understanding (1/2)

In [ ]:
! wget https://storage.googleapis.com/class25jan2022/share/2008.csv

--2022-02-12 15:05:45--  https://storage.googleapis.com/class25jan2022/share/2008.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.112.128, 172.217.212.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.112.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689413344 (657M) [text/csv]
Saving to: ‘2008.csv’

2008.csv            100%[===================>] 657.48M   225MB/s    in 2.9s    

2022-02-12 15:05:48 (225 MB/s) - ‘2008.csv’ saved [689413344/689413344]



In [ ]:
! wc -l ./2008.csv

7009729 ./2008.csv


In [ ]:
! head -3 2008.csv

Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
2008,1,3,4,2003,1955,2211,2225,WN,335,N712SW,128,150,116,-14,8,IAD,TPA,810,4,8,0,,0,NA,NA,NA,NA,NA
2008,1,3,4,754,735,1002,1000,WN,3231,N772SW,128,145,113,2,19,IAD,TPA,810,5,10,0,,0,NA,NA,NA,NA,NA


# Dataproc Only

In [ ]:
#! hdfs dfs -rm -f /user/cloudera/input/2008.csv

In [ ]:
#! hdfs dfs -mkdir -p /user/cloudera/input

In [ ]:
#! hadoop fs -put 2008.csv /user/cloudera/input

In [ ]:
#! hdfs dfs -ls /user/cloudera/input

In [ ]:
#file_path = '/user/cloudera/input/2008.csv'

# Data Understanding (2/2)

In [ ]:
file_path = '2008.csv'

In [ ]:
airline_df = spark.read.format('csv').\
option('header','true').option('mode','DROPMALFORMED')\
.load(file_path)

In [ ]:
airline_df.count()

7009728

In [ ]:
airline_row_df = airline_df

In [ ]:
airline_row_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: string (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

# 3.Data Preparation

In [ ]:
from pyspark.sql.types import *
from pyspark.sql import functions as sparkf

In [ ]:
crunched_df = airline_row_df.\
withColumn('DepTime',sparkf.col('DepTime').\
           cast(DoubleType())).\
withColumn('TaxiOut',sparkf.col('TaxiOut').\
           cast(DoubleType())).\
withColumn('TaxiIn',sparkf.col('TaxiIn').\
           cast(DoubleType())).\
withColumn('DepDelay',sparkf.col('DepDelay').\
           cast(DoubleType())).\
withColumn('DayOfWeek',sparkf.col('DayOfWeek').\
           cast(DoubleType())).\
withColumn('Distance',sparkf.col('Distance').\
           cast(DoubleType())).\
withColumn('ArrDelay',sparkf.col('ArrDelay').\
           cast(DoubleType()))

In [ ]:
crunched_df.printSchema()

root
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- DayofMonth: string (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- CRSDepTime: string (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: string (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: string (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- Cancelled: string (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: string (nullable = true)
 |-- CarrierDelay:

In [ ]:
! pip install pandas

In [ ]:
import pandas as pd
crunched_df.\
              select(['DepTime','TaxiOut','TaxiIn',\
                      'DayOfWeek','Distance','ArrDelay']).\
              describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
DepTime,6873482,1333.8300461105448,478.06889486629836,1.0,2400.0
TaxiOut,6872670,16.453045177492882,11.332798654232155,0.0,429.0
TaxiIn,6858079,6.860851704974527,4.933649371300466,0.0,308.0
DayOfWeek,7009728,3.9241815088973495,1.9882589459851212,1.0,7.0
Distance,7009728,726.3870294253928,562.1018034840403,11.0,4962.0
ArrDelay,6855029,8.16845238729114,38.50193694882867,-519.0,2461.0


In [ ]:
def t_timeperiod(origin):
    if origin is None:
      period = None
    elif origin > 0 and origin < 600:
      period = '00.01-05.59'
    elif origin >=600 and origin <1200:
      period = '06.00-11.59'
    elif origin >=1200 and origin <1800:
      period = '12.00-17.59'
    elif origin >= 1800 and origin <= 2400:
      period = '18.00-24.00'
    else:
      period = 'NA'
    return period

In [ ]:
timeperiod = sparkf.udf(lambda x: t_timeperiod(x),StringType())

In [ ]:
discretized_df = crunched_df.\
withColumn('DepTime',timeperiod(sparkf.col('DepTime')))

In [ ]:
max_distance = discretized_df.select(sparkf.max('Distance')).collect()[0][0]
min_distance = discretized_df.select(sparkf.min('Distance')).collect()[0][0]

In [ ]:
max_ArrDelay = discretized_df.select(sparkf.max('ArrDelay')).collect()[0][0]
min_ArrDelay = discretized_df.select(sparkf.min('ArrDelay')).collect()[0][0]

In [ ]:
def t_normalized_distance(origin):
  if origin is None:
    return None
  else:
    return ((origin-min_distance)/(max_distance-min_distance))

In [ ]:
def t_normalized_ArrDelay(origin):
  if origin is None:
    return None
  else:
    return ((origin-min_ArrDelay)/(max_ArrDelay-min_ArrDelay))

In [ ]:
normalized_distance = sparkf.udf(lambda x: t_normalized_distance(x),DoubleType())

In [ ]:
normalized_ArrDelay = sparkf.udf(lambda x: t_normalized_ArrDelay(x),DoubleType())

In [ ]:
normalized_df = discretized_df.\
withColumn('Distance', normalized_distance(sparkf.col('Distance'))).\
withColumn('ArrDelay', normalized_ArrDelay(sparkf.col('ArrDelay')))

In [ ]:
features_df = normalized_df.\
select(['UniqueCarrier','Origin','Dest',\
        'DepTime','TaxiOut','TaxiIn','DepDelay',\
        'DayOfWeek','Distance','ArrDelay'])

In [ ]:
final_df = features_df.dropna()

In [ ]:
final_df.count()

6855029

In [ ]:
final_df.show()

+-------------+------+----+-----------+-------+------+--------+---------+--------------------+-------------------+
|UniqueCarrier|Origin|Dest|    DepTime|TaxiOut|TaxiIn|DepDelay|DayOfWeek|            Distance|           ArrDelay|
+-------------+------+----+-----------+-------+------+--------+---------+--------------------+-------------------+
|           WN|   IAD| TPA|18.00-24.00|    8.0|   4.0|     8.0|      4.0| 0.16138153908301353|0.16946308724832215|
|           WN|   IAD| TPA|06.00-11.59|   10.0|   5.0|    19.0|      4.0| 0.16138153908301353|0.17483221476510066|
|           WN|   IND| BWI|06.00-11.59|   17.0|   3.0|     8.0|      4.0|  0.1017976166431024|0.17885906040268457|
|           WN|   IND| BWI|06.00-11.59|    7.0|   3.0|    -4.0|      4.0|  0.1017976166431024|0.17214765100671142|
|           WN|   IND| BWI|18.00-24.00|   10.0|   3.0|    34.0|      4.0|  0.1017976166431024| 0.1855704697986577|
|           WN|   IND| JAX|18.00-24.00|   10.0|   4.0|    25.0|      4.0|  0.136

# 4. Modeling (and making some data transformation)

In [ ]:
training_df,test_df = final_df.\
randomSplit([0.90,0.10], seed = 12)

In [ ]:
training_df.count()

6170207

In [ ]:
training_df.printSchema()

root
 |-- UniqueCarrier: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- TaxiOut: double (nullable = true)
 |-- TaxiIn: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- DayOfWeek: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- ArrDelay: double (nullable = true)



In [ ]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder

In [ ]:
DepTimeIndexer = StringIndexer\
(handleInvalid='keep',inputCol='DepTime',outputCol='DepTimeIndexed')

In [ ]:
UniqueCarrierIndexer = StringIndexer\
(handleInvalid='keep',inputCol='UniqueCarrier',\
outputCol='UniqueCarrierIndexed')

In [ ]:
UniqueCarrierOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='UniqueCarrierIndexed',\
outputCol='UniqueCarrierVec')

In [ ]:
OriginIndexer = StringIndexer(handleInvalid='keep',inputCol='Origin',\
                             outputCol='OriginIndexed')

In [ ]:
OriginOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='OriginIndexed',\
outputCol='OriginVec')

In [ ]:
DestIndexer = StringIndexer(handleInvalid='keep',inputCol='Dest',\
                           outputCol='DestIndexed')

In [ ]:
DestOneHotEncoder = OneHotEncoder\
(dropLast=False,inputCol='DestIndexed',\
 outputCol='DestVec')

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

In [ ]:
featureAssembler = VectorAssembler\
(inputCols=['DepTimeIndexed',\
            'UniqueCarrierVec',\
            'OriginVec',\
            'DestVec',\
            'TaxiOut','TaxiIn',\
            'DepDelay',\
            'DayOfWeek',\
            'Distance'
           ], outputCol='***features')

In [ ]:
from pyspark.ml.regression import RandomForestRegressor

In [ ]:
dt = RandomForestRegressor\
(labelCol='ArrDelay',featuresCol='***features')

In [ ]:
pipeline_dt = Pipeline().\
setStages([DepTimeIndexer,\
           UniqueCarrierIndexer,\
           UniqueCarrierOneHotEncoder,\
           OriginIndexer,\
           OriginOneHotEncoder,\
           DestIndexer,\
           DestOneHotEncoder,\
           featureAssembler,dt])

In [ ]:
dtModel = pipeline_dt.fit(training_df)

# 5.Evaluation

In [ ]:
result_df = dtModel.transform(test_df.dropna())

In [ ]:
result_df.select(['ArrDelay','Prediction']).show()

+-------------------+-------------------+
|           ArrDelay|         Prediction|
+-------------------+-------------------+
|0.18053691275167785|0.17404855690356016|
|0.18523489932885906|0.18063408652134083|
|  0.174496644295302| 0.1736258337533351|
|0.17315436241610738| 0.1741502831157759|
|0.16711409395973154|0.17308549833087156|
|0.16946308724832215| 0.1731714156553799|
|0.17885906040268457| 0.1782235578111112|
|0.17651006711409395|0.17308549833087156|
| 0.1691275167785235|0.17308549833087156|
|0.18892617449664428|0.18274329715487742|
|  0.175503355704698|0.17422805431453708|
|0.17248322147651007|0.17320351995347325|
|0.17583892617449665| 0.1743143001371026|
|0.17919463087248322|0.17501851233604154|
|0.22550335570469798| 0.2134472410852767|
|0.18288590604026847|0.18205289756128368|
|0.17651006711409395| 0.1731367470574122|
| 0.1697986577181208| 0.1732544516214766|
|0.17348993288590603| 0.1743106583153643|
|0.18120805369127516|0.18191561582332327|
+-------------------+-------------

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol='prediction',labelCol='ArrDelay',metricName='r2')
print("R Squared (R2) on test data = %g"\
     % lr_evaluator_r2.evaluate(result_df))

R Squared (R2) on test data = 0.713274


In [ ]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol='prediction',labelCol='ArrDelay',metricName='rmse')
print("Root Mean Squared Error (RMSE) on test data = %g"\
     % lr_evaluator_rmse.evaluate(result_df))

Root Mean Squared Error (RMSE) on test data = 0.00695644


# Alternative Models 

Linear Regression

In [ ]:
from pyspark.ml.regression import LinearRegression
dt = LinearRegression\
(labelCol='ArrDelay',featuresCol='***features')
pipeline_dt = Pipeline().\
setStages([DepTimeIndexer,\
           UniqueCarrierIndexer,\
           UniqueCarrierOneHotEncoder,\
           OriginIndexer,\
           OriginOneHotEncoder,\
           DestIndexer,\
           DestOneHotEncoder,\
           featureAssembler,dt])
dtModel = pipeline_dt.fit(training_df)

In [ ]:
result_df = dtModel.transform(test_df.dropna())

In [ ]:
result_df.select(['ArrDelay','Prediction']).show()

+-------------------+-------------------+
|           ArrDelay|         Prediction|
+-------------------+-------------------+
|0.18053691275167785|0.17910804210515174|
|0.18523489932885906|0.18753360852334258|
|  0.174496644295302|0.17397994494268537|
|0.17315436241610738|0.17330149410297396|
|0.16711409395973154| 0.1696761103443279|
|0.16946308724832215| 0.1693116444775552|
|0.17885906040268457| 0.1777435177513773|
|0.17651006711409395|0.16973860303400998|
| 0.1691275167785235| 0.1703896863083193|
|0.18892617449664428| 0.1890280530748225|
|  0.175503355704698|0.17605450355063232|
|0.17248322147651007|  0.171319169099532|
|0.17583892617449665|0.17934190845796996|
|0.17919463087248322| 0.1775777621682953|
|0.22550335570469798|0.22760936661944114|
|0.18288590604026847|0.18121395841754384|
|0.17651006711409395| 0.1715818479931237|
| 0.1697986577181208|0.17159233529349893|
|0.17348993288590603| 0.1723205062647802|
|0.18120805369127516|0.18728012779402972|
+-------------------+-------------

In [ ]:
lr_evaluator_r2 = RegressionEvaluator\
(predictionCol='prediction',labelCol='ArrDelay',metricName='r2')
print("R Squared (R2) on test data = %g"\
     % lr_evaluator_r2.evaluate(result_df))

R Squared (R2) on test data = 0.945077


In [ ]:
lr_evaluator_rmse = RegressionEvaluator\
(predictionCol='prediction',labelCol='ArrDelay',metricName='rmse')
print("Root Mean Squared Error (RMSE) on test data = %g"\
     % lr_evaluator_rmse.evaluate(result_df))

Root Mean Squared Error (RMSE) on test data = 0.00304462
